# $\left\langle L_{B,\tau, Z}^2\right\rangle_{L<L_{\rm lim}}$ Calculation

Here we want to provide the code to calculate $\left\langle L_{B,\tau, Z}^2\right\rangle_{L<L_{\rm lim}}$ for a given set of isochrhones. Since this can be an expensive task, we're going to use Cython to speed up some parts of the code.  Much of this notebook is identical to the notebook which calculates $f_{\rm det}$.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
%load_ext Cython

In [2]:
## a function for evaluating the IMF
def xi_IMF(ms,mpows,mcuts):
    res = np.zeros(ms.shape)
    (normx,normy) = (1.0,1.0)
    cuti = 0
    nms = len(ms)
    for i in range(nms):
        res[i] = normy*((ms[i]/normx)**mpows[cuti])
        if (i!=nms-1) and (ms[i+1]>mcuts[cuti+1]):
            normx = ms[i]
            normy = res[i]
            cuti +=1
    return res

## a function for calculating the IMF normalization
def norm_IMF(mpows,mcuts):
    res = 0
    (normx,normy) = (1.0,1.0)
    nmcs = len(mcuts)
    for i in range(nmcs-1):
        res += normy*(mcuts[i+1]**(mpows[i]+1) - mcuts[i]**(mpows[i]+1))/((normx**mpows[i])*(mpows[i]+1))
        normy = normy*((mcuts[i+1]/normx)**mpows[i])
        normx = mcuts[i+1]
    return res

In [3]:
# First specify the properties of the populations you'd like to calculate f_detect for
# by giving lists of metallicities and ages. These should be linearly spaced 
metlist = np.array([0.5,0.25,0.0,-0.25,-0.5,-0.75,-1.0,-1.25,-1.5,-1.75,-2.0,-2.25,-2.5,-2.75,-3.,-3.25])
agelist = np.array([8.95,9.,9.05,9.1,9.15,9.2,9.25,9.3,9.35,9.4,9.45,9.5,9.55,9.6,9.65,9.7,9.75,9.8,9.85,9.9,9.95,10.,10.05,10.1])
nmet = len(metlist) # number of metallicities
nage = len(agelist) # number of ages
dage = agelist[1] - agelist[0] # logarithmic difference in  age bins, for selection below

## specify directory where isochrones are stored, this is set up for the 
## MIST Roman Isocrones, changing that would involve changing the below
iso_dir = "../../mist_isos/"
## name od output file
output_name = "../data/l2_mist" # it will have a .npy at the end

## define the IMF normalization
## specify the power-law indices for the IMF that you would like to use
## here we define a Kroupa IMF
mpows = [-1.3,-2.3]
mcuts = [0.08,0.5,120]
assert(len(mpows)+1==len(mcuts))
## specify the number of mass samples you would like for integrals of the IMF
nmass_samp = 100000
# mass range in solar masses
(mMin,mMax) = (mcuts[0],mcuts[-1])
# masses spaced linearly throughout imf space
mlin = np.linspace(mMin,mMax,nmass_samp)
# evaluate IMF
xi = xi_IMF(mlin,mpows,mcuts)
# the IMF normalization over the mass range of interest
xim_norm = norm_IMF(mpows,mcuts)
xi = xi/xim_norm

# then specify the range of *absolute* magnitudes you would like to cover
# these are related to the exposure time of the observation and the distance 
# to the observed population through Eq 2 of our paper
nmags = 100
# what are the brightest and faintest magnitudes you want to calculate for
(minMag,maxMag) = (3,-7)
# get the range of absolute magnitudes you would like to calculate fdet for
Mags = np.linspace(minMag,maxMag,nmags)
# specify the number of filters. The default is 7 (R,Z,Y,J,W,H,F). Changing this 
# number will require some hard code changes below
nfilts = 7
# conversion from Vega mags to AB mags for each band
AB_Vega = np.array([0.147,0.485,0.647,0.950,1.012,1.281,1.546]) #This is m_AB - m_Vega

The following cell is where we define the function needed to compute the luminosity fluctuations, which is effectively the only cell which is different from the $f_{\rm det}$ calculation. We first give the Python implementation, and then the Cython implementation.

In [4]:
def get_L2avg_python(B_mu, B_cut, ms, xiofm):
    fsum = 0
    
    for i in range(len(B_mu)-1):
        L21 = 10**(B_mu[i]/-1.25)
        L22 = 10**(B_mu[i+1]/-1.25)
        if((B_mu[i]>B_cut) and (B_mu[i+1] > B_cut)):
            fsum += (L21*xiofm[i] + L22*xiofm[i+1])*(ms[i+1] - ms[i])/2.
        
        elif((B_mu[i]>B_cut) or (B_mu[i+1] > B_cut)):
            aB = (B_mu[i+1] - B_mu[i])/(ms[i+1] - ms[i])
            bB = B_mu[i] - ms[i]*aB
            mcross = (B_cut - bB)/aB
            Bcross = mcross*aB + bB
            L2cross = 10**(Bcross/-1.25)
            aX = (xiofm[i+1] - xiofm[i])/(ms[i+1] - ms[i])
            bX = xiofm[i] - ms[i]*aX
            xicross = mcross*aX + bX
            if(B_mu[i]>B_cut):
                fsum += (L21*xiofm[i] + L2cross*xicross)*(mcross - ms[i])/2.
            else:
                fsum += (L2cross*xicross + L22*xiofm[i+1])*(ms[i+1] - mcross)/2.
    return fsum

In [5]:
%%cython
# cython: boundscheck=False, wraparound=False, nonecheck=False, cdivision=True
import numpy as np
cimport numpy as np


cimport cython

cpdef double get_L2avg(const double[::1] B_mu, const double B_cut, 
                   const double[::1] ms, const double[::1] xiofm, 
                   const int N):
    cdef:
        double fsum = 0
        double aB,bB,aX,bX,mcross,xicross,Bcross
        double L21,L22,L2cross
        int i
    
    for i in range(N-1):
        L21 = 10**(B_mu[i]/-1.25)
        L22 = 10**(B_mu[i+1]/-1.25)
        if((B_mu[i]>B_cut) and (B_mu[i+1] > B_cut)):
            fsum += (L21*xiofm[i] + L22*xiofm[i+1])*(ms[i+1] - ms[i])/2.
        
        elif((B_mu[i]>B_cut) or (B_mu[i+1] > B_cut)):
            aB = (B_mu[i+1] - B_mu[i])/(ms[i+1] - ms[i])
            bB = B_mu[i] - ms[i]*aB
            mcross = (B_cut - bB)/aB
            Bcross = mcross*aB + bB
            L2cross = 10**(Bcross/-1.25)
            aX = (xiofm[i+1] - xiofm[i])/(ms[i+1] - ms[i])
            bX = xiofm[i] - ms[i]*aX
            xicross = mcross*aX + bX
            if(B_mu[i]>B_cut):
                fsum += (L21*xiofm[i] + L2cross*xicross)*(mcross - ms[i])/2.
            else:
                fsum += (L2cross*xicross + L22*xiofm[i+1])*(ms[i+1] - mcross)/2.
    return fsum

In [5]:
%%time
# shape of the total output array
totdat = np.zeros((nmet,nage,nfilts,nmags))
for (l,met) in enumerate(metlist):
    print(met)
    if met>-0.1:
        (age,imass,T,L) = np.loadtxt("%smist_fehp%1.2f.iso.cmd"%(iso_dir,abs(met)),usecols=[1,2,4,6]).T
        (R,Z,Y,J,W,H,F) = np.loadtxt("%smist_fehp%1.2f.iso.cmd"%(iso_dir,abs(met)),usecols=[9,10,11,12,13,14,15]).T
    else:
        (age,imass,T,L) = np.loadtxt("%smist_fehm%1.2f.iso.cmd"%(iso_dir,abs(met)),usecols=[1,2,4,6]).T
        (R,Z,Y,J,W,H,F) = np.loadtxt("%smist_fehm%1.2f.iso.cmd"%(iso_dir,abs(met)),usecols=[9,10,11,12,13,14,15]).T

    tsave = np.zeros((nage,nfilts,nmags))
    # select a single isochrone
    for (k,a) in enumerate(agelist):
        sids = np.intersect1d(np.where(age>a-dage/2),np.where(age<a+dage/2))

        # interpolate what the right magnitudes are
        R_out = R[sids] + AB_Vega[0]
        Z_out = Z[sids] + AB_Vega[1]
        Y_out = Y[sids] + AB_Vega[2]
        J_out = J[sids] + AB_Vega[3]
        W_out = W[sids] + AB_Vega[4]
        H_out = H[sids] + AB_Vega[5]
        F_out = F[sids] + AB_Vega[6]
        mass_sel = imass[sids]
        xim = np.interp(imass[sids],mlin,xi)
        (fRdetect,fZdetect,fYdetect,fJdetect) = (np.zeros(nmags),np.zeros(nmags),np.zeros(nmags),np.zeros(nmags))
        (fWdetect,fHdetect,fFdetect) = (np.zeros(nmags),np.zeros(nmags),np.zeros(nmags))
        for (i,mag) in enumerate(Mags):
            fRdetect[i] = get_L2avg(R_out,mag,mass_sel,xim,len(xim))
            fZdetect[i] = get_L2avg(Z_out,mag,mass_sel,xim,len(xim))
            fYdetect[i] = get_L2avg(Y_out,mag,mass_sel,xim,len(xim))
            fJdetect[i] = get_L2avg(J_out,mag,mass_sel,xim,len(xim))
            fWdetect[i] = get_L2avg(W_out,mag,mass_sel,xim,len(xim))
            fHdetect[i] = get_L2avg(H_out,mag,mass_sel,xim,len(xim))
            fFdetect[i] = get_L2avg(F_out,mag,mass_sel,xim,len(xim))
        tsave[k] = np.array([fRdetect,fZdetect,fYdetect,fJdetect,fWdetect,fHdetect,fFdetect])
    totdat[l] = tsave
np.save(output_name,totdat)

0.5
0.25
0.0
-0.25
-0.5
-0.75
-1.0
-1.25
-1.5
-1.75
-2.0
-2.25
-2.5
-2.75
-3.0
-3.25
CPU times: user 53.5 s, sys: 1.54 s, total: 55 s
Wall time: 55.4 s
